In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
np.random.seed(0)

### Q1. Read the dataset (tab, csv, xls, txt, inbuilt dataset)

In [ ]:
data=pd.read_csv('/kaggle/input/indian-liver-patient-records/indian_liver_patient.csv')
data.head()

### Q2. Summarize important observations from the data set.

* We have 10 independent variables to predict the class (y-variable). The class contains binary values either one or two. From the question description it is clear that the notation one stands for non-liver patients and two stands for people with liver disease.
* We have data of 416 people with liver problem and 167 people without liver problem, which we can find either from the question or by taking the value count of the class variable.

In [ ]:
data['Dataset'].value_counts()

In [ ]:
data.info()

* Out of 10 independent variables there is only one variable which is categorical, we have to convert this into numerical variable by mapping or we have to encode them using Label encoding or One-Hot encoding.

In [ ]:
print(data.isnull().sum())

* There are only four data points in Albumin/Globulin ratio (A/G) which are null values. Depends on the importance or the data validity we can either drop them since they might contribute a small portion of information for the analysis or we can impute them using different parameters within the variable.

In [ ]:
data.shape

* The data set consists of 11 information columns(10-X-variable and 1 Y-variable) and 583 data rows.

In [ ]:
data.describe().T

* From the statistical table we can say that there are alot of outliers in the data, which has to be further treated.

In [ ]:
print('Categorical Columns are ','\n',list(data.select_dtypes(include='object')),'\n')
print('Numerical Columns are ','\n',list(data.select_dtypes(exclude='object')))

### Q3. Check for defects in the data. Perform necessary actions to ‘fix’ these defects.

In [ ]:
data.isnull().mean()*100

* The data consists of only 0.68% null-values in A/G column with respect to the values which specified in the column.

#### Null-Value treatment on A/G:

In [ ]:
data['Albumin_and_Globulin_Ratio']=data['Albumin_and_Globulin_Ratio'].fillna(data['Albumin_and_Globulin_Ratio'].mean()) 
     # In this case the distribution is not very much skewed thus we can either go for median or mode.

#### Conversion of Gender column to numerical column.

In [ ]:
data['Gender']=data['Gender'].map({'Male':0,'Female':1})

#### Before Outlier Treatment.

In [ ]:
data.drop('Dataset',axis=1).plot(kind='box',layout=(2,5),subplots=True,figsize=(12,6))
plt.show()

* From the graph it is clear that, as mentioned before there are alot of outliers in the data provided for most of the variables. I am not checking outliers for target variable.

In [ ]:
data.describe().T

* On the basis of the statistics table we can impute a few of the variables which exhibit huge outlier values.Eg : Alkaline_Phosphotase, Alamine_Aminotransferase, etc.
    The values in these columns lies mostly within hundreds where as the maximum value is in the range of e^+03.

In [ ]:
q1=data['Total_Bilirubin'].quantile(0.25)
q3=data['Total_Bilirubin'].quantile(0.75)
iqr=q3-q1
data['Total_Bilirubin']=data['Total_Bilirubin'].apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x).apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x)


q1=data['Direct_Bilirubin'].quantile(0.25)
q3=data['Direct_Bilirubin'].quantile(0.75)
iqr=q3-q1
data['Direct_Bilirubin']=data['Direct_Bilirubin'].apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x).apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x)

q1=data['Alkaline_Phosphotase'].quantile(0.25)
q3=data['Alkaline_Phosphotase'].quantile(0.75)
iqr=q3-q1
data['Alkaline_Phosphotase']=data['Alkaline_Phosphotase'].apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x).apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x)


q1=data['Alamine_Aminotransferase'].quantile(0.25)
q3=data['Alamine_Aminotransferase'].quantile(0.75)
iqr=q3-q1
data['Alamine_Aminotransferase']=data['Alamine_Aminotransferase'].apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x).apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x)


q1=data['Aspartate_Aminotransferase'].quantile(0.25)
q3=data['Aspartate_Aminotransferase'].quantile(0.75)
iqr=q3-q1
data['Aspartate_Aminotransferase']=data['Aspartate_Aminotransferase'].apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x).apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x)


q1=data['Total_Protiens'].quantile(0.25)
q3=data['Total_Protiens'].quantile(0.75)
iqr=q3-q1
data['Total_Protiens']=data['Total_Protiens'].apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x).apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x)


q1=data['Albumin_and_Globulin_Ratio'].quantile(0.25)
q3=data['Albumin_and_Globulin_Ratio'].quantile(0.75)
iqr=q3-q1
data['Albumin_and_Globulin_Ratio']=data['Albumin_and_Globulin_Ratio'].apply(lambda x : q3+(1.5*iqr) if x>q3+(1.5*iqr) else x).apply(lambda x : q1-(1.5*iqr) if x<q1-(1.5*iqr) else x)



# Capping the outlier values to the quantile (+-) 1.5*iqr value.

#### NOTE : Capping of the data may not be a good solution in this problem for outlier treatment, because TB, DB and such parameters are very much significant for the detection or prediction of Liver Disease, due to time limit and lack of domain knowledge in the field, I would like to proceed further with capping the outliers with the Quantile + or - 1.5 x IQR value for slightly better result that imputing with statistical values like mean,median or mode.

### After Outlier Treatment

In [ ]:
data.drop('Dataset',axis=1).plot(kind='box',layout=(2,5),subplots=True,figsize=(12,6))
plt.show()

#### Checking the distribution of target variable (Class)

In [ ]:
sns.distplot(data['Dataset'],rug=True,color='midnightblue') 

* From the distribution curve, we can see that there is a difference in density that both the values show different local maximas, thus there is a bias existing between the class labels, which has to be rectified before going further with the model building and prediction.

In [ ]:
sns.countplot(data['Dataset'])
print('The ratio between the target variables are ',data['Dataset'].value_counts()[2]/data['Dataset'].value_counts()[1])

* The ratio is between Label : 2 (People without liver disease) and Label-1 (People with liver disease) is 0.40 or in other words there are about 60% of entries in the dataset about people with liver disease than the data about the people with liver disease.
* This is an example of a biased data, for further rectification we have to equate this ratio to one, which stands for the equal distribution of both the class variables.
* We can perform up-resampling, down-resampling methods, since the data size is small, down sampling will not give a good result. I would like to proceed with Synthetic creation of datapoints from the lower value class to make upto higher value class by continues iteration using "SMOTE" Technique.

### Q4. Summarize relationships among variables (10 marks)

* To check for the relationship between variables we can consider heatmap of correlation matrix.

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(data.corr(),cmap='magma',annot=True)
plt.show()

* From the visualisation we can say the highest correlation is with A/G with correlation of 0.18. But we cannot take this conclusion because in this caseClass variables are highly biased. And we can say the number of people which is considered for this data collection is mostly Male category, thus we cannot say a solid understanding out of this heatmap.

In [ ]:
data.corr()['Dataset'] # To get exact values of correlation by X-Variables on Y-Variable.

#### To check for the association between variables, we can consider scatter plotting for the visualisation.

In [ ]:
for i in data.columns:
    sns.scatterplot(data[i],data['Dataset'],color='midnightblue')
    plt.show()

#### The scatter plot gives a good insight about the dependancy of class variable over different other variables in the data frame. They are not actually linear, they follow a sigmoid curve. The values of class variable dependancy will be either 1 or 2, that is the reason why the scatter plot follows such a pattern.

In [ ]:
plt.figure(figsize=(10,5))
data.corr()['Dataset'].sort_values(ascending=False).plot(kind='bar',color='black')
plt.xticks(rotation=45)
plt.xlabel('Variables in the Data')
plt.ylabel('Correlation Values')
plt.show()

* The above graph shows the variables correlation with class variables

### Q5. Split dataset into train and test (70:30)

In [ ]:
x=data.drop('Dataset',axis=1)
y=data['Dataset']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=43)

In [ ]:
x.describe()

In [ ]:
x_train.describe()

In [ ]:
x_test.describe()

#### From the above tables, we can say that the train test split is a very good representation of the data since the values of statistical parameters like Mean and Standard Deviations are more or less similar in the population ('Dataset' provided) and the Splitted Data (Train-Test Data).

### Q6. Fit a base model and explain the reason of selecting that model. Please write your key observations.

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

In [ ]:
ss.fit(x_train)

In [ ]:
x_train=ss.transform(x_train)
x_test=ss.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score,recall_score
log=LogisticRegression()

In [ ]:
log.fit(x_train,y_train)

In [ ]:
y_pred=log.predict(x_test)
accuracy_score(y_test,y_pred)

* In the base model, the accuracy is around 75%, over all if we conisder the accuracy score in the data we can say that it is quite good, but the data provided is biased data, thus we will get a better picture only when we check for the confusion matrix of the data prediction. 

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='d')

* From the visualisation of the confusion matrix we can say that the model prediction were not good enough because in this case Type -II error is 47. Which is not acceptable at any cost in the health care industry, So the primary aim of the prediction will be for reducing Type - II error and make it upto True Predictions.

In [ ]:
print('F1 Score of the prediction is ',f1_score(y_test,y_pred))
print('Recall Score of the prediction is ',recall_score(y_test,y_pred))

* This is not a good prediction due to the bias in the dataset. Thus I would like to end up with this process and proceed further by first eliminating the data bias using SMOTE Technique and use a slighty complex model for the analysis.

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
#no of features
nof_list=np.arange(1,19)            
high_score=0
#Variable to store the optimum features
nof=0          
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 43)
    model = LogisticRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))
rfe = RFE(model,3)
rfe.fit_transform(X_train,y_train)
y_pred = rfe.predict(X_test)

accuracy_score(y_test,y_pred)
pd.DataFrame(rfe.support_,index=x.columns)

#### From the RFE Evaluation it is found that 3 variables are mostly important for the detection of liver disease or the prediction of class variable. But, Since the size of the data is small, we should consider a different algorithm for the calculation.

## Decision Tree Model for Classification.

* Decision Tree will tends to over fit in the small dataset, thus I am considering this as to be my base model. Then proceed further by eliminating the varience error or over fitting by using a ensembling method for the analysis. Here, treating of bias is the first step.

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()

In [ ]:
x=data.drop('Dataset',axis=1)
y=data['Dataset']

In [ ]:
x_01=x
y_01=y
# For reusability of the variable

In [ ]:
x,y=smote.fit_sample(x,y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=43)

In [ ]:
ss.fit(x_train)

In [ ]:
x_train=ss.transform(x_train)
x_test=ss.transform(x_test)

In [ ]:
pd.DataFrame(y)['Dataset'].value_counts() 

* This shows that the smote function made the Y-variable to be in the ratio of 1.0 or a normal distribution by avoiding the bias in the data.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()

In [ ]:
dt.fit(x_train,y_train)

In [ ]:
y_pred=dt.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

* By using the decision Tree also, I could't make the prediction as I expected. Thus going for a different model might give a better result.

In [ ]:
pd.DataFrame(dt.feature_importances_,index=x_01.columns,columns=['Feature Importance']).sort_values(by='Feature Importance',ascending=False).T

* So, From these values it is clear that the model consideration is more on a few features for the prediction. Which are aranged in the order.

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
#Cohen-Kappa Score for Statistical Analysis.
cohen_kappa_score(y_test,y_pred)

### Q7. How do you improve the accuracy of the model? Write clearly the changes that you will make before re-fitting the model. Fit the final model.

>Points to be noted :
    >> * The data is biased thus we did perform SMOTE in the previous step even then the other models were not good enough for the predictions..<br/>
    >> * In this case we have to go for a slightly advanced model calculation. <br/>
    >> * I would like to proceed further with ensemble modeling. (Random Forest (To avoid over fitting caused by decision tree) <br/>
    >> * The accuracy of base line model is not upto the expectation, thus proceed for slightly complex model and then tune hyper parameters accordingly

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

In [ ]:
rf.fit(x_train,y_train)

In [ ]:
y_pred=rf.predict(x_test)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
cohen_kappa_score(y_test,y_pred)

#### In this even though the accuracy is high, the confusion matrix is not upto what we are looking for. So, we can briefly say that Decision Tree is giving pretty good result, but it can be because of the over fitting of the data on a low amount of data.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV #Due to comptational capability picking Randomised Search over Grid Search

In [ ]:
param={'n_estimators':range(1,50),'random_state':range(1,10),'criterion':['gini','entropy'],'min_samples_split':range(2,10),'min_samples_leaf':range(2,10)}

In [ ]:
rand=RandomizedSearchCV(estimator=rf,param_distributions=param,return_train_score=True,cv=5,random_state=3,scoring='recall')

In [ ]:
rand.fit(x_train,y_train)

In [ ]:
y_pred=rand.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
cohen_kappa_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='d')
plt.show()

#### I am considering this to be the final model I could make with the time limits and computational limitations. I used Randomized Search CV for the hyper parametric tuning where the lack of computational power and time limit did not permit me to proceed with Grid Search CV for a slightly better accuracy.

### Qn.8 Summary

* The model which I used for the final submission is Random Forest Classifier, even though the Decision Tree can make a slighly better model, I strongly assume that it is due to over fitting of the data due to small dataset. Decision Tree can be very much handy once we use it in a larger dataset. We have different models like KNN,Guassian Naive-Bayes,Bagging Techniques, Boosting Techniques, Extra Tree Classifier and much more. I limit the analysis here due to lack of different parameters and strongly believe further analysis might help for reducing Type-II error.

* In the final model, the true values are in the same order as it is given in the question, where TN is low and TP is high, that is the way the analysis should be, because the data provided has large data of people with liver disease and 40% data are less for the data about the people without liver disease. Thus,  the analysis is in the right track for the prediction of liver disease from other 10 parameters.

#### The data provided is a data of health care industry, the good prediction of disease will be useful in industries like health care or in the field of insurance companies we can use. The feature what my model used for the predictions are as follow:

In [ ]:
pd.DataFrame(rf.feature_importances_,index=x_01.columns).sort_values(by=0,ascending=False).T

In [ ]:
pd.DataFrame(rf.feature_importances_,index=x_01.columns).sort_values(by=0,ascending=False).plot(kind='bar',color='midnightblue',figsize=(10,6))

* From these parameters we can say the first few features the model considered with higher importance for the prediction.

#### Model complexity optimization and hyper-parametric tuning are the factors which effects the model accuracy where as removing or optimisation of Bias in the target variable is the key factor which played a crucial role in the final-model building.

* Even then the numbers which it considered as Type-II error still stands, thus I the model should be further improvised, I count this as the risk factor in the analysis.

# END.!